In [24]:
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import classification_report, confusion_matrix

from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


In [25]:
# Load
df = pd.read_csv('/content/drive/MyDrive/5G_NIDD_multiclass_clean.csv', low_memory=False)

print("Original shape:", df.shape)

# Target
y = df['Label']
X = df.drop(columns=['Label', 'Attack Type', 'Attack Tool'], errors='ignore')

# Remove obvious non-learning columns
drop_cols = [
    'SrcMac','DstMac','SrcAddr','DstAddr','StartTime','LastTime',
    'SrcOui','DstOui'
]

X = X.drop(columns=[c for c in drop_cols if c in X.columns], errors='ignore')

# Keep only numeric features
X = X.select_dtypes(include=[np.number])

print("After numeric selection:", X.shape)


Original shape: (1215890, 112)
After numeric selection: (1215890, 86)


In [26]:
X.replace([np.inf, -np.inf], np.nan, inplace=True)
X.fillna(0, inplace=True)


In [27]:
selector = SelectKBest(score_func=f_classif, k=36)
X_selected = selector.fit_transform(X, y)

selected_features = X.columns[selector.get_support()]
print("Selected Features:", selected_features.tolist())


/usr/local/lib/python3.12/dist-packages/sklearn/feature_selection/_univariate_selection.py:111: UserWarning: Features [ 1  7 13 14 19 20 21 22 23 24 25 26 27 34 35 36 37 48 49 50 51 57 58 59
 60 61 62 63 64 65 66 67 68 69 70 71 72 77 78 79 80] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.12/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


Selected Features: ['Rank', 'Seq', 'Dur', 'RunTime', 'Mean', 'Sum', 'Min', 'Max', 'sTos', 'dTos', 'sTtl', 'dTtl', 'sHops', 'dHops', 'TotPkts', 'SrcPkts', 'DstPkts', 'TotBytes', 'SrcBytes', 'DstBytes', 'Offset', 'sMeanPktSz', 'dMeanPktSz', 'Loss', 'SrcLoss', 'DstLoss', 'pLoss', 'SrcWin', 'DstWin', 'sVid', 'dVid', 'SrcTCPBase', 'DstTCPBase', 'TcpRtt', 'SynAck', 'AckDat']


In [28]:
le = LabelEncoder()
y_encoded = le.fit_transform(y)

num_classes = len(np.unique(y_encoded))
print("Classes:", num_classes)


Classes: 20


In [29]:
X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y_encoded,
    test_size=0.2,
    stratify=y_encoded,
    random_state=42
)


In [30]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)   # FIT ONLY TRAIN
X_test  = scaler.transform(X_test)        # TRANSFORM TEST


In [31]:
X_train = X_train.reshape(-1, 36, 1)
X_test  = X_test.reshape(-1, 36, 1)


In [32]:
def MobileNetV1(drop_rate=0.5, dense_units=256, width_mult=1.0):

    inp = Input(shape=(36,1))

    x = Reshape((36,1,1))(inp)

    # Initial conv
    x = Conv2D(int(32*width_mult),(3,3),padding="same")(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # Depthwise Block 1
    x = DepthwiseConv2D((3,3),padding="same")(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(int(64*width_mult),(1,1),padding="same")(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # Depthwise Block 2
    x = DepthwiseConv2D((3,3),padding="same")(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(int(128*width_mult),(1,1),padding="same")(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # Pooling
    x = GlobalAveragePooling2D()(x)

    # Classifier
    x = Dense(dense_units, activation="relu")(x)
    x = Dense(dense_units//2, activation="relu")(x)
    x = Dropout(drop_rate)(x)

    out = Dense(num_classes, activation="softmax")(x)

    model = Model(inp, out)
    return model


In [33]:
!pip install keras-tuner


In [34]:
import tensorflow as tf

def focal_loss(gamma=2.0, alpha=0.25):

    def loss(y_true, y_pred):

        y_true = tf.cast(y_true, tf.int32)
        y_true = tf.one_hot(y_true, depth=num_classes)

        ce = tf.keras.losses.categorical_crossentropy(y_true, y_pred)

        pt = tf.exp(-ce)
        focal = alpha * tf.pow(1 - pt, gamma) * ce

        return tf.reduce_mean(focal)

    return loss


In [35]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

classes = np.unique(y_train)

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=classes,
    y=y_train
)

class_weights = dict(zip(classes, class_weights))
print(class_weights)


{np.int64(0): np.float64(0.64811172410117), np.int64(1): np.float64(0.6491671115856914), np.int64(2): np.float64(3.325965944060726), np.int64(3): np.float64(4.20650406504065), np.int64(4): np.float64(37.819284603421465), np.int64(5): np.float64(44.74296228150874), np.int64(6): np.float64(1.3619983757596124), np.int64(7): np.float64(4.309374446216552), np.int64(8): np.float64(5.309563318777292), np.int64(9): np.float64(5.274438781043271), np.int64(10): np.float64(1.9601644365629534), np.int64(11): np.float64(4.803516049382716), np.int64(12): np.float64(5.220652640618291), np.int64(13): np.float64(5.217292426517915), np.int64(14): np.float64(1.5948189926547744), np.int64(15): np.float64(1.9197000197355436), np.int64(16): np.float64(0.12998123867505493), np.int64(17): np.float64(0.21242149215139894), np.int64(18): np.float64(6.053721682847897), np.int64(19): np.float64(5.8995147986414365)}


In [ ]:
import keras_tuner as kt
from tensorflow.keras.callbacks import EarlyStopping

def build_model(hp):

    model = MobileNetV1(
        drop_rate = hp.Choice("dropout",[0.3,0.5,0.6]),
        dense_units = hp.Choice("dense",[128,256,512]),
        width_mult = hp.Choice("width",[0.75,1.0,1.25])
    )

    lr = hp.Choice("lr",[1e-2,1e-3,1e-4])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        loss=focal_loss(gamma=2, alpha=0.25),
        metrics=["accuracy"]
    )

    return model



tuner = kt.Hyperband(
    build_model,
    objective="val_accuracy",
    max_epochs=7,
    factor=3,
    directory="tuning_mobilenet",
    project_name="5g_mobilenet_only"
)

# ---- Tune on subset ----
sample_idx = np.random.choice(len(X_train), size=int(len(X_train)*1), replace=False)
X_tune = X_train[sample_idx]
y_tune = y_train[sample_idx]

stop_early = EarlyStopping(monitor='val_loss', patience=3)

tuner.search(
    X_tune, y_tune,
    validation_split=0.2,
    epochs=10,
    batch_size=512,
    callbacks=[stop_early],
    verbose=1
)

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print("Best Hyperparameters:")
print(best_hps.values)


In [14]:
model = tuner.hypermodel.build(best_hps)

history = model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=15,
    batch_size=512,
    class_weight=class_weights,
    callbacks=[
        EarlyStopping(patience=8, restore_best_weights=True),
        ReduceLROnPlateau(patience=4)
    ]
)


Epoch 1/10
1710/1710 ━━━━━━━━━━━━━━━━━━━━ 42s 20ms/step - accuracy: 0.6853 - loss: 0.1020 - val_accuracy: 0.7890 - val_loss: 0.0915 - learning_rate: 0.0100
Epoch 2/10
1710/1710 ━━━━━━━━━━━━━━━━━━━━ 28s 15ms/step - accuracy: 0.8352 - loss: 0.0461 - val_accuracy: 0.8441 - val_loss: 0.0407 - learning_rate: 0.0100
Epoch 3/10
1710/1710 ━━━━━━━━━━━━━━━━━━━━ 25s 15ms/step - accuracy: 0.8491 - loss: 0.0402 - val_accuracy: 0.8244 - val_loss: 0.0522 - learning_rate: 0.0100
Epoch 4/10
1710/1710 ━━━━━━━━━━━━━━━━━━━━ 25s 15ms/step - accuracy: 0.8555 - loss: 0.0375 - val_accuracy: 0.8525 - val_loss: 0.0386 - learning_rate: 0.0100
Epoch 5/10
1710/1710 ━━━━━━━━━━━━━━━━━━━━ 26s 15ms/step - accuracy: 0.8582 - loss: 0.0362 - val_accuracy: 0.8596 - val_loss: 0.0445 - learning_rate: 0.0100
Epoch 6/10
1710/1710 ━━━━━━━━━━━━━━━━━━━━ 26s 15ms/step - accuracy: 0.8601 - loss: 0.0357 - val_accuracy: 0.8575 - val_loss: 0.0402 - learning_rate: 0.0100
Epoch 7/10
1710/1710 ━━━━━━━━━━━━━━━━━━━━ 25s 15ms/step - accura

In [15]:
pred = model.predict(X_test)
pred = np.argmax(pred, axis=1)

print(classification_report(y_test, pred))


7600/7600 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step
              precision    recall  f1-score   support

           0       0.99      0.95      0.97     18761
           1       0.90      0.93      0.92     18730
           2       0.51      0.86      0.64      3656
           3       0.73      0.79      0.76      2890
           4       0.40      0.41      0.41       322
           5       0.29      0.20      0.23       271
           6       0.92      0.91      0.92      8927
           7       0.84      0.38      0.53      2822
           8       0.98      0.88      0.93      2290
           9       0.99      0.87      0.93      2305
          10       0.82      0.68      0.74      6203
          11       0.45      0.56      0.50      2531
          12       0.39      0.17      0.23      2329
          13       0.50      0.75      0.60      2331
          14       0.84      0.93      0.88      7624
          15       1.00      0.81      0.89      6334
          16       0.97      0.86    

In [41]:
import shutil, os
import keras_tuner as kt
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

# remove previous runs
if os.path.exists("tuning_mobilenet"):
    shutil.rmtree("tuning_mobilenet")

def build_model(hp):

    model = MobileNetV1(
        drop_rate = hp.Float("dropout", 0.2, 0.7, step=0.1),
        dense_units = hp.Int("dense", 128, 512, step=64),
        width_mult = hp.Float("width", 0.5, 1.5, step=0.25)
    )

    lr = hp.Float("lr", 1e-5, 1e-2, sampling="log")

    model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)
    return model

tuner = kt.Hyperband(
    build_model,
    objective="val_accuracy",
    max_epochs=7,
    factor=3,
    directory="tuning_mobilenet",
    project_name="5g_mobilenet_only"
)

# ---- Tune on subset ----
sample_idx = np.random.choice(len(X_train), size=int(len(X_train)*1), replace=False)
X_tune = X_train[sample_idx]
y_tune = y_train[sample_idx]

stop_early = EarlyStopping(monitor='val_loss', patience=3)

tuner.search(
    X_tune, y_tune,
    validation_split=0.2,
    epochs=10,
    batch_size=512,
    callbacks=[stop_early],
    verbose=1
)

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print("Best Hyperparameters:")
print(best_hps.values)

Trial 10 Complete [00h 02m 10s]
val_accuracy: 0.8091784119606018

Best val_accuracy So Far: 0.8747732043266296
Total elapsed time: 00h 21m 20s
Best Hyperparameters:
{'dropout': 0.30000000000000004, 'dense': 448, 'width': 0.75, 'lr': 0.009658312491068482, 'tuner/epochs': 7, 'tuner/initial_epoch': 3, 'tuner/bracket': 1, 'tuner/round': 1, 'tuner/trial_id': '0000'}


In [42]:
model = tuner.hypermodel.build(best_hps)

history = model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=15,
    batch_size=512,
    class_weight=class_weights,
    callbacks=[
        EarlyStopping(patience=8, restore_best_weights=True),
        ReduceLROnPlateau(patience=4)
    ]
)


Epoch 1/15
1710/1710 ━━━━━━━━━━━━━━━━━━━━ 38s 18ms/step - accuracy: 0.5970 - loss: 1.4896 - val_accuracy: 0.7904 - val_loss: 0.5285 - learning_rate: 0.0097
Epoch 2/15
1710/1710 ━━━━━━━━━━━━━━━━━━━━ 25s 15ms/step - accuracy: 0.8087 - loss: 0.8543 - val_accuracy: 0.4121 - val_loss: 3.5513 - learning_rate: 0.0097
Epoch 3/15
1710/1710 ━━━━━━━━━━━━━━━━━━━━ 25s 15ms/step - accuracy: 0.8189 - loss: 0.7657 - val_accuracy: 0.7741 - val_loss: 0.6440 - learning_rate: 0.0097
Epoch 4/15
1710/1710 ━━━━━━━━━━━━━━━━━━━━ 25s 15ms/step - accuracy: 0.8477 - loss: 0.7075 - val_accuracy: 0.8459 - val_loss: 0.3785 - learning_rate: 0.0097
Epoch 5/15
1710/1710 ━━━━━━━━━━━━━━━━━━━━ 25s 15ms/step - accuracy: 0.8516 - loss: 0.6871 - val_accuracy: 0.8569 - val_loss: 0.3296 - learning_rate: 0.0097
Epoch 6/15
1710/1710 ━━━━━━━━━━━━━━━━━━━━ 25s 15ms/step - accuracy: 0.8564 - loss: 0.6594 - val_accuracy: 0.8453 - val_loss: 0.4622 - learning_rate: 0.0097
Epoch 7/15
1710/1710 ━━━━━━━━━━━━━━━━━━━━ 25s 15ms/step - accura

In [43]:
pred = model.predict(X_test)
pred = np.argmax(pred, axis=1)

print(classification_report(y_test, pred))


7600/7600 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step
              precision    recall  f1-score   support

           0       0.97      0.96      0.97     18761
           1       0.97      0.88      0.92     18730
           2       0.32      0.45      0.37      3656
           3       0.31      0.94      0.47      2890
           4       0.22      0.68      0.34       322
           5       0.21      0.20      0.20       271
           6       0.97      0.68      0.80      8927
           7       0.71      0.63      0.67      2822
           8       0.99      0.87      0.93      2290
           9       0.94      0.88      0.91      2305
          10       0.83      0.60      0.70      6203
          11       0.45      0.77      0.57      2531
          12       0.48      0.58      0.53      2329
          13       0.48      0.36      0.41      2331
          14       0.96      0.89      0.92      7624
          15       0.98      0.93      0.96      6334
          16       0.99      0.84    

In [44]:
model = tuner.hypermodel.build(best_hps)

history = model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=25,
    batch_size=512,
    class_weight=class_weights,
    callbacks=[
        EarlyStopping(patience=8, restore_best_weights=True),
        ReduceLROnPlateau(patience=4)
    ]
)


Epoch 1/25
1710/1710 ━━━━━━━━━━━━━━━━━━━━ 40s 18ms/step - accuracy: 0.6029 - loss: 1.4194 - val_accuracy: 0.6781 - val_loss: 1.0280 - learning_rate: 0.0097
Epoch 2/25
1710/1710 ━━━━━━━━━━━━━━━━━━━━ 25s 15ms/step - accuracy: 0.8318 - loss: 0.7886 - val_accuracy: 0.6242 - val_loss: 1.9196 - learning_rate: 0.0097
Epoch 3/25
1710/1710 ━━━━━━━━━━━━━━━━━━━━ 25s 15ms/step - accuracy: 0.8416 - loss: 0.7304 - val_accuracy: 0.6996 - val_loss: 1.0300 - learning_rate: 0.0097
Epoch 4/25
1710/1710 ━━━━━━━━━━━━━━━━━━━━ 26s 15ms/step - accuracy: 0.8538 - loss: 0.6740 - val_accuracy: 0.6786 - val_loss: 2.2844 - learning_rate: 0.0097
Epoch 5/25
1710/1710 ━━━━━━━━━━━━━━━━━━━━ 25s 15ms/step - accuracy: 0.8514 - loss: 0.6626 - val_accuracy: 0.8478 - val_loss: 0.3574 - learning_rate: 0.0097
Epoch 6/25
1710/1710 ━━━━━━━━━━━━━━━━━━━━ 26s 15ms/step - accuracy: 0.8582 - loss: 0.6405 - val_accuracy: 0.8536 - val_loss: 0.3375 - learning_rate: 0.0097
Epoch 7/25
1710/1710 ━━━━━━━━━━━━━━━━━━━━ 25s 15ms/step - accura

In [45]:
pred = model.predict(X_test)
pred = np.argmax(pred, axis=1)

print(classification_report(y_test, pred))


7600/7600 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step
              precision    recall  f1-score   support

           0       0.99      0.97      0.98     18761
           1       0.99      0.90      0.94     18730
           2       0.41      0.63      0.49      3656
           3       0.50      0.83      0.62      2890
           4       0.18      0.87      0.30       322
           5       0.17      0.27      0.21       271
           6       0.99      0.76      0.86      8927
           7       0.77      0.62      0.69      2822
           8       0.98      0.87      0.93      2290
           9       0.99      0.87      0.93      2305
          10       0.89      0.62      0.73      6203
          11       0.35      0.88      0.50      2531
          12       0.45      0.55      0.49      2329
          13       0.53      0.43      0.48      2331
          14       0.89      0.93      0.91      7624
          15       0.98      0.88      0.93      6334
          16       0.99      0.85    